In [ ]:

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:

def load_images_and_masks(base_path, classes=('benign', 'malignant'), img_size=(256, 256)):
    img_list, mask_list, label_list = [], [], []

    for class_name in classes:
        folder = os.path.join(base_path, class_name)
        for fname in os.listdir(folder):
            if 'mask' in fname.lower():
                continue
            img_path = os.path.join(folder, fname)
            mask_path = os.path.join(folder, fname.split('.')[0] + '_mask.png')
            if not os.path.exists(mask_path): continue

            img = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if img is None or mask is None:
                continue

            img = cv2.resize(img, img_size).astype('float32') / 255.0
            mask = cv2.resize(mask, img_size).astype('float32') / 255.0
            mask = np.expand_dims(mask, axis=-1)

            img_list.append(img)
            mask_list.append(mask)
            label_list.append(class_name)

    return np.array(img_list), np.array(mask_list), np.array(label_list)


In [ ]:

base_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"
X_seg, y_seg, labels_seg = load_images_and_masks(base_path, img_size=(256, 256))

X_train_seg, X_val_seg, y_train_seg, y_val_seg = train_test_split(
    X_seg, y_seg, test_size=0.2, random_state=42, stratify=labels_seg)


In [ ]:

def build_model(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=inputs)
    encoder_output = base_model.output

    x = Conv2DTranspose(256, (3,3), strides=(2,2), padding="same")(encoder_output)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(128, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(64, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(32, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(16, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    outputs = Conv2D(1, (1,1), activation="sigmoid")(x)

    return Model(inputs, outputs)

model_seg = build_model()
model_seg.compile(optimizer=Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])
model_seg.summary()


In [ ]:

history = model_seg.fit(
    X_train_seg, y_train_seg,
    validation_data=(X_val_seg, y_val_seg),
    epochs=50,
    batch_size=16,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)


In [ ]:

def plot_sample(image, predicted_mask, actual_mask, index):
    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].imshow(image)
    ax[0].set_title(f'Original Image {index}')
    ax[0].axis('off')

    ax[1].imshow(predicted_mask, cmap='viridis')
    ax[1].set_title(f'Predicted Mask {index}')
    ax[1].axis('off')

    ax[2].imshow(actual_mask, cmap='viridis')
    ax[2].set_title(f'Actual Mask {index}')
    ax[2].axis('off')
    plt.tight_layout()
    plt.show()

predictions = model_seg.predict(X_val_seg)
predictions = (predictions > 0.5).astype(np.uint8)

for i in range(3):
    plot_sample(X_val_seg[i], predictions[i].squeeze(), y_val_seg[i].squeeze(), i)
